## Aim

Write a general ABC sampler, to understand the technique and to apply it to exoplanet occurrence problems.

In [1]:
import numpy as np
import scipy
import scipy.stats as stats
import functools
import sys
sys.path.append('..')
from dev.abc_generic import ABCSampler

In [2]:
# First simple example: a Gaussian fit you can do with MLE.
# Suppose we've sampled data from N(50, 10), but our best guess is it's N(N(40, 20), N(11, 2)).
prior = stats.multivariate_normal(np.array([40, 11]), np.diag([20, 2]))
def candidate_getter(p):
    def candidate(size):
        return np.random.normal(p[0], np.abs(p[1]), size)
    return candidate

gaussian_sampler = ABCSampler(prior, candidate_getter)
gaussian_sampler.sample(np.random.normal(50, 10, (100,)), verbose=False)

array([49.22133487, 11.63676804])

In [3]:
# and now, PMC!
# let's do the same exact problem as in the first one
gaussian_sampler.sample_pmc(np.random.normal(50, 10, (100,)), [10, 5, 1, 0.5, 0.1, 0.01, 1e-3, 1e-4, 1e-5], 7)

Attempting to clear threshold 10
Attempting to clear threshold 5


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
# Exp(22) but we estimate Exp(max(0, N(21, 2)))
prior = stats.halfnorm(21, 2)
def candidate_getter(p):
    def candidate(size):
        return np.random.exponential(p, size)
    return candidate

exp_sampler = ABCSampler(prior, candidate_getter)
exp_sampler.sample(np.random.exponential(22, (100,)), threshold=0.001, verbose=False)

1. https://arxiv.org/pdf/1802.09720.pdf